In [1]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel, TfidfModel

In [3]:
df = pd.read_csv('/content/drive/MyDrive/2023년 10월-11월 프로젝트/토큰화 진행 및 불용어 처리/2021년 기사(완료).csv')
df.drop(columns=['Unnamed: 0'],inplace=True)
df.reset_index(drop=True, inplace=True)
df

,date,url,tokenize_title,tokenize_content
0,2021-01-01T03:37:00,https://news.naver.com/main/read.nhn?mode=LS2D...,"['네이버', '카카오', '추천', '뉴스', '사실', '여부', '보다', '...","['네이버', '부터', '자체', '개발', '콘텐츠', '추천', '시스템', ..."
1,2021-01-01T08:00:00,https://news.naver.com/main/read.nhn?mode=LS2D...,"['주식', '부호', '클럽', '명', '김범수', '카카오', '의장', '순...","['코스피', '사상', '최고', '마감', '주식', '부자', '늘어', '고..."
2,2021-01-01T13:04:00,https://news.naver.com/main/read.nhn?mode=LS2D...,"['클럽', '명', '카카오', '김범수', '늘', '고', '아모레', '서'...","['코스피', '지수', '사상', '최고', '치', '마감', '하면서', '이..."
3,2021-01-01T13:21:00,https://news.naver.com/main/read.nhn?mode=LS2D...,"['테크', '체인', '저', '네이버', '보다', '카카오', '일상', '바...","['네이버', '카카오', '응답', '들', '우리', '일상', '바꿀', '기..."
4,2021-01-01T17:05:00,https://news.naver.com/main/read.nhn?mode=LS2D...,"['작년', '매출', '신기록', '쓴', '카카오', '엔씨', '올해', '좋다']","['증권사', '망치', '분석', '곳', '중', '곳', '최대', '실적',..."
...,...,...,...,...
8422,2021-12-30T14:49:00,https://news.naver.com/main/read.naver?mode=LS...,"['뉴스', '픽', '카카오', '지그재그', '첫', '패션', '임박']","['카카오', '온라인', '패션', '플랫폼', '지그재그', '자체', '브랜드..."
8423,2021-12-30T16:33:00,https://news.naver.com/main/read.naver?mode=LS...,"['상생', '외친', '카카오', '모빌리티', '일', '성', '아닌', '지...","['업계', '의견', '수렴', '해', '장기', '상생', '구현', '목표'..."
8424,2021-12-30T18:02:00,https://news.naver.com/main/read.naver?mode=LS...,"['택시', '기사', '업계', '공존', '카카오', '모빌리티', '상생', ...","['대리', '운전', '수수료', '개선', '이어', '새해', '상생', '플..."
8425,2021-12-31T14:03:00,https://news.naver.com/main/read.naver?mode=LS...,"['새로운', '골프', '시장', '개척', '게임', '체인', '저', '카카...","['코로나', '사태', '계기', '골프', '산업', '세대', '뿐', '아니..."


In [4]:
# 각 'tokenize_title'과 'tokenize_content'의 type이 str이므로 이를 list로 변환하기 위함
def str_list(token):
  str_list = token
  str_list = str_list.replace("[","").replace("]","").replace("'","").replace(" ","")
  result = str_list.split(",")
  return result

In [5]:
# title에 대한 LDA모델 적용
title_docs = []
for doc in df['tokenize_title']:
  title_docs.append(str_list(doc))
title_docs

[['네이버', '카카오', '추천', '뉴스', '사실', '여부', '보다', '많은', '클릭', '받을', '기사', '골라'],
 ['주식', '부호', '클럽', '명', '김범수', '카카오', '의장', '순위', '껑충'],
 ['클럽', '명', '카카오', '김범수', '늘', '고', '아모레', '서', '경배', '폭락'],
 ['테크', '체인', '저', '네이버', '보다', '카카오', '일상', '바꿀', '기업', '설문', '결과'],
 ['작년', '매출', '신기록', '쓴', '카카오', '엔씨', '올해', '좋다'],
 ['큰', '장', '선다', '크래프톤', '카카오', '형제', '출격'],
 ['카뱅', '케뱅', '새해', '테크', '핀', '공격', '경영', '빅뱅'],
 ['기다리면', '무료', '재미', '본', '카카오', '영상', '반대', '유료'],
 ['낡은', '규', '제혁', '파', '된', '녹슨', '칼', '네이버', '카카오', '규제', '하겠다는', '정부'],
 ['땅', '좁다', '플라잉', '카', '띄우겠다는', '카카오', '독점', '깰까'],
 ['카카오', '손잡고', '세대', '공략'],
 ['카카오', '페이', '디지털', '손해', '보험사', '예비', '신청'],
 ['카카오', '페이', '디지털', '손해', '보험사', '예비', '신청'],
 ['카카오', '페이', '보험사', '설립', '속도', '예비', '신청', '완료'],
 ['카카오', '페이', '디지털', '손해', '보험사', '예비', '신청'],
 ['카카오', '페이', '디지털', '손해', '보험사', '설립', '예비', '신청'],
 ['카카오', '페이', '디지털', '손', '보사', '설립', '한다', '예비', '신청', '완료'],
 ['카카오', '페이', '디지털', '손해', '보험사', '예비', '신청', '완료'],
 ['카카

In [6]:
id2word_title = corpora.Dictionary(title_docs)
print(id2word_title)

Dictionary<8043 unique tokens: ['골라', '기사', '네이버', '뉴스', '많은']...>


In [7]:
for value in id2word_title:
  print(value, id2word_title[value])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
3043 포인트
3044 피자
3045 섯거
3046 기다려
3047 손잡은
3048 점화
3049 태운
3050 디쉬
3051 칼날
3052 홈쇼핑
3053 뭔가
3054 뭘
3055 스트리트
3056 있겠니
3057 서학
3058 열전
3059 근로
3060 쉽게
3061 유적지
3062 밀크
3063 블록
3064 사이드
3065 만났다
3066 노려
3067 베드
3068 테스트
3069 펫
3070 노
3071 믹스
3072 뉴욕증시
3073 훈풍
3074 부활
3075 독보
3076 잘나가네
3077 전세계
3078 선전
3079 나스닥
3080 주인공
3081 어떨까
3082 나흘
3083 치솟은
3084 만발
3085 질렀다
3086 스테이지
3087 신한은행
3088 파이브
3089 했어요
3090 돈나무
3091 웃는
3092 선착순
3093 한경
3094 쟁
3095 위반
3096 혐의
3097 협상
3098 밀고
3099 바라본다
3100 들썩
3101 소문
3102 정도
3103 교보
3104 추정
3105 가능했나
3106 개강
3107 공매
3108 아카데미
3109 온비드
3110 캠코
3111 건의
3112 국토부
3113 제출
3114 철회
3115 대책
3116 하
3117 도대체
3118 몇개
3119 수십
3120 챙긴다
3121 파는
3122 거위
3123 낳는
3124 막
3125 팬덤
3126 하이브
3127 압박
3128 촉구
3129 모멘텀
3130 유예
3131 지그재그
3132 발굴
3133 신인
3134 개국
3135 방영
3136 단숨
3137 위로
3138 의류
3139 갔지만
3140 싸운다
3141 쉼
3142 품는다
3143 바
3144 포기
3145 향방
3146 시점
3147 나타날
3148 분가
3149 치도
3150 치솟는
3151 봉투
3152 송금
3153 돌린
3154 국제
3155 표준
3156 환경
3157 획득
3158

In [8]:
corpus_TDM = []
for doc in title_docs:
  result = id2word_title.doc2bow(doc)
  corpus_TDM.append(result)

corpus_TDM # index, 등장 횟수 (순서는 index 순으로 sorting해서 출력됨)

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1)],
 [(10, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1)],
 [(10, 1),
  (12, 1),
  (14, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1)],
 [(2, 1),
  (6, 1),
  (10, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1)],
 [(10, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)],
 [(10, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1)],
 [(33, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1)],
 [(10, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1)],
 [(2, 1),
  (10, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1)],
 [(10, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1)],
 [(10, 1), (78, 1), (79, 1), (80, 1)],
 [(10, 1)

In [9]:
# 토큰화 진행 및 불용어 처리에서 불용어에 대해 업데이트 해줘야함!!!
tfidf = TfidfModel(corpus_TDM)
corpus_TFIDF = tfidf[corpus_TDM]

n = 10 # 토픽갯수 - 변경가능
lda = LdaModel(corpus=corpus_TFIDF, id2word=id2word_title, num_topics=n, random_state=100)

for t in lda.print_topics():
  print(t[0],":",t[1])

0 : 0.018*"대표" + 0.018*"버전" + 0.014*"체제" + 0.012*"올인" + 0.012*"답변" + 0.009*"사람" + 0.008*"브레인" + 0.007*"의장" + 0.007*"김범수" + 0.006*"물류"
1 : 0.018*"페이" + 0.018*"상장" + 0.017*"코스피" + 0.011*"마감" + 0.010*"입성" + 0.010*"배그" + 0.010*"대표이사" + 0.009*"이재용" + 0.007*"신규" + 0.007*"연동"
2 : 0.029*"상생" + 0.013*"페이" + 0.009*"하락" + 0.009*"증시" + 0.008*"전환" + 0.008*"퀵" + 0.007*"주차장" + 0.007*"지원" + 0.007*"코스피" + 0.007*"네이버"
3 : 0.019*"미래" + 0.018*"모빌리티" + 0.018*"투자" + 0.013*"버스" + 0.013*"메타" + 0.011*"캘린더" + 0.011*"김범수" + 0.010*"게임" + 0.009*"출연" + 0.009*"규제"
4 : 0.016*"페이" + 0.015*"렌터카" + 0.014*"디지털" + 0.012*"활동" + 0.012*"포토" + 0.011*"본인" + 0.010*"기능" + 0.009*"카겜" + 0.009*"신청" + 0.008*"반등"
5 : 0.026*"센터" + 0.023*"고발" + 0.021*"탈세" + 0.015*"집중" + 0.014*"미래" + 0.012*"김범수" + 0.011*"게임" + 0.011*"이니셔티브" + 0.010*"택시" + 0.010*"대표"
6 : 0.015*"대표" + 0.013*"오딘" + 0.012*"라이브" + 0.011*"생태계" + 0.011*"커머스" + 0.011*"시즌" + 0.009*"그" + 0.008*"출사표" + 0.008*"공동" + 0.007*"라운드"
7 : 0.013*"네이버" + 0.011*"예약" + 0.010*"앱" + 0.009*"거대" 

In [10]:
# 반복문 돌려서 기사에 토픽 일일이 배정해줘야 함(title 부분)
lda[corpus_TFIDF[0]]

[(0, 0.023898784),
 (1, 0.023874879),
 (2, 0.0238766),
 (3, 0.023883436),
 (4, 0.023886716),
 (5, 0.023915289),
 (6, 0.02387474),
 (7, 0.6067094),
 (8, 0.10094044),
 (9, 0.12513971)]

In [11]:
for i in range(len(corpus_TFIDF)):
  print(lda[corpus_TFIDF[i]])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[(0, 0.26878583), (1, 0.04282768), (2, 0.04282502), (3, 0.0428365), (4, 0.38861287), (5, 0.042824533), (6, 0.0428229), (7, 0.04282577), (8, 0.042817276), (9, 0.042821664)]
[(0, 0.037679557), (1, 0.037689675), (2, 0.03768468), (3, 0.03768567), (4, 0.6608276), (5, 0.03768261), (6, 0.03768025), (7, 0.037706714), (8, 0.03768376), (9, 0.03767949)]
[(0, 0.029968087), (1, 0.029990237), (2, 0.03000522), (3, 0.02997302), (4, 0.5330676), (5, 0.029970866), (6, 0.22711687), (7, 0.029971732), (8, 0.02996882), (9, 0.029967614)]
[(0, 0.03198492), (1, 0.031988844), (2, 0.10634964), (3, 0.032005582), (4, 0.03198889), (5, 0.031984366), (6, 0.031984363), (7, 0.03200424), (8, 0.031988353), (9, 0.6377208)]
[(0, 0.04058049), (1, 0.43193173), (2, 0.040584117), (3, 0.040596347), (4, 0.24341923), (5, 0.0405766), (6, 0.040575795), (7, 0.04058357), (8, 0.040576976), (9, 0.040575106)]
[(0, 0.030481415), (1, 0.72564954), (2, 0.030487917), (3, 0.03048497), (4, 0.03048506), (5, 0.

In [12]:
for i in range(len(corpus_TFIDF)):
  dominant_topic = sorted(lda[corpus_TFIDF[i]], key=lambda x: x[1], reverse=True)[0]
  df.loc[i, 'topic(title)'] = dominant_topic[0]
  df.loc[i, 'topic_details(title)'] = dominant_topic[1]
df

,date,url,tokenize_title,tokenize_content,topic(title),topic_details(title)
0,2021-01-01T03:37:00,https://news.naver.com/main/read.nhn?mode=LS2D...,"['네이버', '카카오', '추천', '뉴스', '사실', '여부', '보다', '...","['네이버', '부터', '자체', '개발', '콘텐츠', '추천', '시스템', ...",7.0,0.606472
1,2021-01-01T08:00:00,https://news.naver.com/main/read.nhn?mode=LS2D...,"['주식', '부호', '클럽', '명', '김범수', '카카오', '의장', '순...","['코스피', '사상', '최고', '마감', '주식', '부자', '늘어', '고...",9.0,0.645294
2,2021-01-01T13:04:00,https://news.naver.com/main/read.nhn?mode=LS2D...,"['클럽', '명', '카카오', '김범수', '늘', '고', '아모레', '서'...","['코스피', '지수', '사상', '최고', '치', '마감', '하면서', '이...",4.0,0.666192
3,2021-01-01T13:21:00,https://news.naver.com/main/read.nhn?mode=LS2D...,"['테크', '체인', '저', '네이버', '보다', '카카오', '일상', '바...","['네이버', '카카오', '응답', '들', '우리', '일상', '바꿀', '기...",9.0,0.558865
4,2021-01-01T17:05:00,https://news.naver.com/main/read.nhn?mode=LS2D...,"['작년', '매출', '신기록', '쓴', '카카오', '엔씨', '올해', '좋다']","['증권사', '망치', '분석', '곳', '중', '곳', '최대', '실적',...",6.0,0.517670
...,...,...,...,...,...,...
8422,2021-12-30T14:49:00,https://news.naver.com/main/read.naver?mode=LS...,"['뉴스', '픽', '카카오', '지그재그', '첫', '패션', '임박']","['카카오', '온라인', '패션', '플랫폼', '지그재그', '자체', '브랜드...",6.0,0.419291
8423,2021-12-30T16:33:00,https://news.naver.com/main/read.naver?mode=LS...,"['상생', '외친', '카카오', '모빌리티', '일', '성', '아닌', '지...","['업계', '의견', '수렴', '해', '장기', '상생', '구현', '목표'...",2.0,0.498462
8424,2021-12-30T18:02:00,https://news.naver.com/main/read.naver?mode=LS...,"['택시', '기사', '업계', '공존', '카카오', '모빌리티', '상생', ...","['대리', '운전', '수수료', '개선', '이어', '새해', '상생', '플...",5.0,0.686084
8425,2021-12-31T14:03:00,https://news.naver.com/main/read.naver?mode=LS...,"['새로운', '골프', '시장', '개척', '게임', '체인', '저', '카카...","['코로나', '사태', '계기', '골프', '산업', '세대', '뿐', '아니...",3.0,0.482271


In [13]:
# 토픽별 비중 확인
df['topic(title)'].unique()

array([7., 9., 4., 6., 1., 8., 0., 3., 5., 2.])

In [14]:
df['topic(title)'].value_counts()

3.0    1226
4.0    1093
2.0     946
7.0     918
5.0     899
8.0     749
1.0     713
0.0     685
9.0     682
6.0     516
Name: topic(title), dtype: int64

In [15]:
# content에 대한 LDA모델 적용
content_docs = []
for doc in df['tokenize_content']:
  content_docs.append(str_list(doc))
content_docs

Output hidden; open in https://colab.research.google.com to view.

In [16]:
id2word_content = corpora.Dictionary(content_docs)
print(id2word_content)

Dictionary<43538 unique tokens: ['가짜', '각', '갖지', '같은', '개발']...>


In [17]:
for value in id2word_content:
  print(value, id2word_content[value])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
38538 특공대
38539 형상
38540 부당하단
38541 빼놓고
38542 늘리라
38543 뽑아라
38544 뽑으라
38545 실직자
38546 쏟아졌었다
38547 쟁의
38548 해직
38549 달려든
38550 벌어졌기
38551 줬는지에
38552 치켜세웠던
38553 심신
38554 내려가면
38555 모자라다
38556 불공평하다는
38557 주물
38558 힘써야
38559 꿨던
38560 내려놓기로
38561 뺏는다는
38562 잊히고
38563 잡다가
38564 국부
38565 자금지원
38566 구준엽
38567 그라피티
38568 나연
38569 낙서
38570 리듬
38571 빌리지
38572 서태지
38573 아트웍
38574 와와
38575 팝아트
38576 현진영
38577 두려워하고
38578 떼기에도
38579 몸살
38580 버리기로
38581 각주
38582 따져서
38583 딱딱하지만
38584 묶여있어서
38585 민법
38586 바뀌어도
38587 벌칙
38588 부르지만
38589 살펴봐
38590 용어인데요
38591 유사해요
38592 장황하게
38593 재미없고
38594 줍의
38595 풀리기
38596 맷집
38597 바꾸려하다
38598 부르겠다고
38599 빚기
38600 뽑자마자
38601 알려져있
38602 전관
38603 줬어야
38604 휘청대고
38605 내렸던
38606 나주시
38607 되어줄지는
38608 바꿀까
38609 비보
38610 술렁이고
38611 싸늘하기만
38612 족벌
38613 개점휴업
38614 뛰어넘고자
38615 부침에서
38616 시급해
38617 익숙함을
38618 친근함
38619 특권
38620 합쳤으며
38621 그려내면서
38622 나날
38623 담아내기
38624 담아내야
38625 망해
38626 보내던
38627 손수호
38628 심성
38629 어린아이로
38630 열도
3863

In [18]:
corpus_TDM = []
for doc in content_docs:
  result = id2word_content.doc2bow(doc)
  corpus_TDM.append(result)

corpus_TDM # index, 등장 횟수 (순서는 index 순으로 sorting해서 출력됨)

Output hidden; open in https://colab.research.google.com to view.

In [19]:
# 토큰화 진행 및 불용어 처리에서 불용어에 대해 업데이트 해줘야함!!!
tfidf = TfidfModel(corpus_TDM)
corpus_TFIDF = tfidf[corpus_TDM]

n = 10 # 토픽갯수 - 변경가능
lda = LdaModel(corpus=corpus_TFIDF, id2word=id2word_content, num_topics=n, random_state=100)

for t in lda.print_topics():
  print(t[0],":",t[1])

0 : 0.002*"외국인" + 0.001*"매도" + 0.001*"포인트" + 0.001*"하락" + 0.001*"지수" + 0.001*"매수" + 0.001*"코스닥" + 0.001*"상위" + 0.001*"기관" + 0.001*"상승"
1 : 0.001*"예스" + 0.001*"뱅크" + 0.000*"발인" + 0.000*"별세" + 0.000*"주가" + 0.000*"씨" + 0.000*"증권" + 0.000*"상장" + 0.000*"장례식" + 0.000*"분할"
2 : 0.002*"백신" + 0.002*"" + 0.002*"진료" + 0.001*"접종" + 0.001*"데이즈" + 0.001*"체인지" + 0.001*"오디오" + 0.001*"연애" + 0.001*"예약" + 0.000*"건강"
3 : 0.008*"택시" + 0.005*"이니셔티브" + 0.005*"보험" + 0.005*"모빌리티" + 0.004*"렌터카" + 0.003*"상생" + 0.003*"가맹" + 0.003*"호출" + 0.003*"센터" + 0.003*"보험사"
4 : 0.006*"인증서" + 0.005*"접종" + 0.004*"백신" + 0.004*"인증" + 0.003*"신용" + 0.002*"대출" + 0.002*"질병" + 0.002*"연말정산" + 0.002*"예약" + 0.002*"전자서명"
5 : 0.002*"블록" + 0.001*"체인" + 0.001*"클레이튼" + 0.001*"배지" + 0.001*"라운드" + 0.001*"테크놀로지" + 0.001*"화폐" + 0.001*"리움" + 0.001*"클립" + 0.001*"게임"
6 : 0.005*"게임" + 0.003*"대표" + 0.003*"페이" + 0.003*"미래" + 0.002*"센터" + 0.002*"주가" + 0.002*"네이버" + 0.002*"콘텐츠" + 0.002*"증권" + 0.002*"사업"
7 : 0.001*"무스" + 0.001*"우마" + 0.001*"게임" + 0.001*"더비

In [20]:
# 반복문 돌려서 기사에 토픽 일일이 배정해줘야 함(content 부분)
lda[corpus_TFIDF[0]]

[(0, 0.013464734),
 (1, 0.013464684),
 (2, 0.013464746),
 (3, 0.37439692),
 (4, 0.013465573),
 (5, 0.0134647535),
 (6, 0.5178839),
 (7, 0.013464669),
 (8, 0.013465243),
 (9, 0.013464772)]

In [21]:
for i in range(len(corpus_TFIDF)):
  print(lda[corpus_TFIDF[i]])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[(0, 0.012807702), (1, 0.012807247), (2, 0.012807006), (3, 0.012809191), (4, 0.012807099), (5, 0.012807087), (6, 0.8847333), (7, 0.012807037), (8, 0.012807263), (9, 0.012807059)]
[(0, 0.012550468), (1, 0.012550255), (2, 0.012549932), (3, 0.012565533), (4, 0.012550013), (5, 0.012550624), (6, 0.8870329), (7, 0.0125499815), (8, 0.012550231), (9, 0.012549998)]
[(0, 0.014389031), (1, 0.014388497), (2, 0.014388363), (3, 0.0143937), (4, 0.014389787), (5, 0.014388467), (6, 0.8704882), (7, 0.014388428), (8, 0.014396571), (9, 0.0143889375)]
[(6, 0.8389818), (8, 0.08309024)]
[(0, 0.01247011), (1, 0.0124700535), (2, 0.012468428), (3, 0.012515265), (4, 0.012469225), (5, 0.012468514), (6, 0.88773125), (7, 0.012470203), (8, 0.012468689), (9, 0.012468282)]
[(6, 0.9222241)]
[(6, 0.91136265)]
[(6, 0.91710114)]
[(3, 0.05995469), (6, 0.87132293)]
[(3, 0.1295455), (6, 0.8010416)]
[(6, 0.931528)]
[(0, 0.010806532), (1, 0.01094236), (2, 0.01081916), (3, 0.010857711), (4, 0

In [22]:
for i in range(len(corpus_TFIDF)):
  dominant_topic = sorted(lda[corpus_TFIDF[i]], key=lambda x: x[1], reverse=True)[0]
  df.loc[i, 'topic(content)'] = dominant_topic[0]
  df.loc[i, 'topic_details(content)'] = dominant_topic[1]
df

,date,url,tokenize_title,tokenize_content,topic(title),topic_details(title),topic(content),topic_details(content)
0,2021-01-01T03:37:00,https://news.naver.com/main/read.nhn?mode=LS2D...,"['네이버', '카카오', '추천', '뉴스', '사실', '여부', '보다', '...","['네이버', '부터', '자체', '개발', '콘텐츠', '추천', '시스템', ...",7.0,0.606472,6.0,0.518082
1,2021-01-01T08:00:00,https://news.naver.com/main/read.nhn?mode=LS2D...,"['주식', '부호', '클럽', '명', '김범수', '카카오', '의장', '순...","['코스피', '사상', '최고', '마감', '주식', '부자', '늘어', '고...",9.0,0.645294,6.0,0.699942
2,2021-01-01T13:04:00,https://news.naver.com/main/read.nhn?mode=LS2D...,"['클럽', '명', '카카오', '김범수', '늘', '고', '아모레', '서'...","['코스피', '지수', '사상', '최고', '치', '마감', '하면서', '이...",4.0,0.666192,6.0,0.547852
3,2021-01-01T13:21:00,https://news.naver.com/main/read.nhn?mode=LS2D...,"['테크', '체인', '저', '네이버', '보다', '카카오', '일상', '바...","['네이버', '카카오', '응답', '들', '우리', '일상', '바꿀', '기...",9.0,0.558865,6.0,0.760719
4,2021-01-01T17:05:00,https://news.naver.com/main/read.nhn?mode=LS2D...,"['작년', '매출', '신기록', '쓴', '카카오', '엔씨', '올해', '좋다']","['증권사', '망치', '분석', '곳', '중', '곳', '최대', '실적',...",6.0,0.517670,6.0,0.923066
...,...,...,...,...,...,...,...,...
8422,2021-12-30T14:49:00,https://news.naver.com/main/read.naver?mode=LS...,"['뉴스', '픽', '카카오', '지그재그', '첫', '패션', '임박']","['카카오', '온라인', '패션', '플랫폼', '지그재그', '자체', '브랜드...",6.0,0.419291,6.0,0.792236
8423,2021-12-30T16:33:00,https://news.naver.com/main/read.naver?mode=LS...,"['상생', '외친', '카카오', '모빌리티', '일', '성', '아닌', '지...","['업계', '의견', '수렴', '해', '장기', '상생', '구현', '목표'...",2.0,0.498462,6.0,0.545828
8424,2021-12-30T18:02:00,https://news.naver.com/main/read.naver?mode=LS...,"['택시', '기사', '업계', '공존', '카카오', '모빌리티', '상생', ...","['대리', '운전', '수수료', '개선', '이어', '새해', '상생', '플...",5.0,0.686084,3.0,0.465864
8425,2021-12-31T14:03:00,https://news.naver.com/main/read.naver?mode=LS...,"['새로운', '골프', '시장', '개척', '게임', '체인', '저', '카카...","['코로나', '사태', '계기', '골프', '산업', '세대', '뿐', '아니...",3.0,0.482271,6.0,0.912868


In [23]:
# 토픽별 비중 확인
df['topic(content)'].unique()

array([6., 3., 4., 8., 7., 2., 1., 9.])

In [24]:
df['topic(content)'].value_counts()

6.0    7172
3.0    1149
4.0      47
8.0      24
1.0      16
2.0      14
7.0       4
9.0       1
Name: topic(content), dtype: int64

In [24]:
# CSV 파일로 저장
csv_filename = "news_2021.csv"
df.to_csv(csv_filename, index=False)